In [36]:
import h5py
import torch
import random
import numpy as np
import pandas as pd
import torchmetrics
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as F
from utils import *
from copy import deepcopy
from lora_layers import LoraConv, LoraLinear
import torch.nn as nn

In [37]:
TRAIN_IMAGES_PATH = 'train.h5'
VAL_IMAGES_PATH = 'val.h5'
TEST_IMAGES_PATH = 'test.h5'
SEED = 0
random.seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Working on {device}.')

Working on cpu.


In [38]:
image_size=(98,98)
mean = [0.7439, 0.5892, 0.7210] 
std = [0.1717, 0.2065, 0.1664]

train_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.Normalize(mean=mean, std=std),
])

val_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.Normalize(mean=mean, std=std)
])



train_dataset = BaselineDataset(TRAIN_IMAGES_PATH, train_transform, 'train')
val_dataset = BaselineDataset(VAL_IMAGES_PATH, val_transform, 'train')

In [39]:
BATCH_SIZE=64
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=BATCH_SIZE)

In [40]:
feature_extractor = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14').to(device)

for name, param in feature_extractor.named_parameters():
    if  "blocks.11" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False



Using cache found in /Users/adil/.cache/torch/hub/facebookresearch_dinov2_main


In [42]:
from models import HistoClassifierHead

classif = HistoClassifierHead(dim_input=384, hidden_dim=64, dropout=0.2)
model=nn.Sequential(feature_extractor,classif)

model.to(device)

OPTIMIZER = 'AdamW'
OPTIMIZER_PARAMS = {'lr': 5e-4, 'weight_decay' : 0.02}
LOSS = 'BCELoss'
METRIC = 'Accuracy'
NUM_EPOCHS = 100
PATIENCE = 15
optimizer = getattr(torch.optim, OPTIMIZER)(model.parameters(), **OPTIMIZER_PARAMS)
criterion = getattr(torch.nn, LOSS)()
metric = getattr(torchmetrics, METRIC)('binary')
min_loss, best_epoch = float('inf'), 0

train_model(model, train_dataloader, val_dataloader, device, optimizer_name=OPTIMIZER, optimizer_params=OPTIMIZER_PARAMS, 
                 loss_name=LOSS, metric_name=METRIC, num_epochs=NUM_EPOCHS, patience=PATIENCE, save_path='best_model_dino_last_block.pth')

KeyboardInterrupt: 

In [ ]:

model.load_state_dict(torch.load('best_model_dino_last_block.pth', weights_only=True))
model.eval()
model.to(device)
prediction_dict = {}

with h5py.File(TEST_IMAGES_PATH, 'r') as hdf:
    test_ids = list(hdf.keys())
     

solutions_data = {'ID': [], 'Pred': []}
with h5py.File(TEST_IMAGES_PATH, 'r') as hdf:
    for test_id in tqdm(test_ids):
        img = np.array(hdf.get(test_id).get('img'))
        img = val_transform(torch.tensor(img)).unsqueeze(0).float()
        pred = model(feature_extractor(img.to(device))).detach().cpu()
        solutions_data['ID'].append(int(test_id))
        solutions_data['Pred'].append(int(pred.item() > 0.5))
solutions_data = pd.DataFrame(solutions_data).set_index('ID')
solutions_data.to_csv('results_2.csv')

  1%|          | 615/85054 [00:12<28:47, 48.88it/s]


KeyboardInterrupt: 